In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
df = pd.read_excel(r"/FastFood_clean_output.xlsx",sheet_name="data")
df_AW = df.loc[df['brand'] == "Harvey's"]
df_AW_loyal = df_AW[['loyal','brand','gender','age_range','children','income']]

In [ ]:
unique_gender = df_AW_loyal['gender'].unique()
for gender in unique_gender:
    stats.probplot(df_AW_loyal[df_AW_loyal['gender'] == gender]['loyal'], dist="norm", plot=plt)
    plt.title("Probability Plot - " +  gender)
    plt.show()

In [ ]:
model = ols('loyal ~ gender + children + income + age_range', data=df_AW_loyal).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)

In [ ]:
ratio1 = df_AW_loyal.groupby('income').std().max() / df_AW_loyal.groupby('income').std().min()
ratio2 = df_AW_loyal.groupby('age_range').std().max() / df_AW_loyal.groupby('age_range').std().min()
ratio3 = df_AW_loyal.groupby('children').std().max() / df_AW_loyal.groupby('children').std().min()
ratio4 = df_AW_loyal.groupby('gender').std().max() / df_AW_loyal.groupby('gender').std().min()
print (ratio1,ratio2,ratio3,ratio4)

In [ ]:
# Create ANOVA backbone table
data = [['Between Groups', '', '', '', '', '', ''], ['Within Groups', '', '', '', '', '', ''], ['Total', '', '', '', '', '', '']] 
anova_table = pd.DataFrame(data, columns = ['Source of Variation', 'SS', 'df', 'MS', 'F', 'P-value', 'F crit']) 
anova_table.set_index('Source of Variation', inplace = True)

# calculate SSTR and update anova table
x_bar = df_AW_loyal['loyal'].mean()
SSTR = df_AW_loyal.groupby('income').count() * (df_AW_loyal.groupby('income').mean() - x_bar)**2
anova_table['SS']['Between Groups'] = SSTR['loyal'].sum()

# calculate SSE and update anova table
SSE = (df_AW_loyal.groupby('income').count() - 1) * df_AW_loyal.groupby('income').std()**2
anova_table['SS']['Within Groups'] = SSE['loyal'].sum()

# calculate SSTR and update anova table
SSTR = SSTR['loyal'].sum() + SSE['loyal'].sum()
anova_table['SS']['Total'] = SSTR

# update degree of freedom
anova_table['df']['Between Groups'] = df_AW_loyal['income'].nunique() - 1
anova_table['df']['Within Groups'] = df_AW_loyal.shape[0] - df_AW_loyal['income'].nunique()
anova_table['df']['Total'] = df_AW_loyal.shape[0] - 1

# calculate MS
anova_table['MS'] = anova_table['SS'] / anova_table['df']

# calculate F 
F = anova_table['MS']['Between Groups'] / anova_table['MS']['Within Groups']
anova_table['F']['Between Groups'] = F

# p-value
anova_table['P-value']['Between Groups'] = 1 - stats.f.cdf(F, anova_table['df']['Between Groups'], anova_table['df']['Within Groups'])

# F critical 
alpha = 0.05
# possible types "right-tailed, left-tailed, two-tailed"
tail_hypothesis_type = "two-tailed"
if tail_hypothesis_type == "two-tailed":
    alpha /= 2
anova_table['F crit']['Between Groups'] = stats.f.ppf(1-alpha, anova_table['df']['Between Groups'], anova_table['df']['Within Groups'])

# Final ANOVA Table
print("ANOVA income")
anova_table

In [ ]:
print("Approach 1: The p-value approach to hypothesis testing in the decision rule")
conclusion = "Failed to reject the null hypothesis."
if anova_table['P-value']['Between Groups'] <= alpha:
    conclusion = "Null Hypothesis is rejected."
print("F-score is:", anova_table['F']['Between Groups'], " and p value is:", anova_table['P-value']['Between Groups'])    
print(conclusion)
    
# The critical value approach
print("\n--------------------------------------------------------------------------------------")
print("Approach 2: The critical value approach to hypothesis testing in the decision rule")
conclusion = "Failed to reject the null hypothesis."
if anova_table['F']['Between Groups'] > anova_table['F crit']['Between Groups']:
    conclusion = "Null Hypothesis is rejected."
print("F-score is:", anova_table['F']['Between Groups'], " and critical value is:", anova_table['F crit']['Between Groups'])
print(conclusion)

In [ ]:
import statsmodels.formula.api as smf
# Fit the regression model
model = smf.ols('CBBE ~ gender + age_range + income + children', data=df_AW).fit()

# Print the model summary
print(model.summary())